In [5]:
from GABRIEL.combined_assistant import CombinedAssistant
import os

api_key = os.getenv('OPENAI_API_KEY')

In [11]:
from GABRIEL.archangel import Archangel

In [12]:
archangel = Archangel()
texts = ['This is a funny story' , "This is a sad story","John broke his leg",'Archangel is a great tool','Archangel is a bad tool']
attributes = ['happy, sad, angry, surprised, grateful']
attribute_dict = {'happy':'happiness refers to joy and positivity','sad':'sad refers to sadness'}
archangel.rate_texts(texts, attributes_dict= attribute_dict, save_folder = '/Users/elliottmokski/Downloads/', file_name = 'archangel_test.csv')

  0%|          | 0/5 [00:00<?, ?it/s]

Here are the parameters for this run:

{
    "model": "gpt-3.5-turbo",
    "n_parallel": 50,
    "num_runs": 10,
    "rate_first": false,
    "temperature": 0.8,
    "timeout": 75,
    "truncate_len": 9500,
    "seed": null,
    "truncate": true,
    "format": "json"
}
The output file will be saved at: /Users/elliottmokski/Downloads//archangel_test.csv
Estimated cost: 0.0


100%|██████████| 5/5 [00:01<00:00,  4.21it/s]


,Text,happy,sad
0,Archangel is a bad tool,0,100
1,John broke his leg,0,80
2,This is a sad story,0,100
3,This is a funny story,60,40
4,Archangel is a great tool,50,0


In [3]:
params = {
    'object_category': '', 'attribute_category': '', 'api_key': api_key,
    'descriptions': ['happiness refers to joy and positivity', 'sad refers to sadness'],
    'attributes': ['happy', 'sad'], 'model': 'gpt-3.5-turbo', 'temperature': 0.8,
    'use_classification': False, 'format': 'json', 'project_probs': False, 'truncate': True,
    'seed': None, 'timeout': 75, 'truncate_len': 9500, 'classification_clarification':'','text':'John broke his foot'
}

assistant = CombinedAssistant(api_key=api_key)
output_df = assistant.rate_single_text(**params)

The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


In [6]:
from GABRIEL import foundational_functions

# Determine the path to the 'Prompts' folder dynamically
package_dir = os.path.dirname(os.path.abspath(foundational_functions.__file__))
templates_dir = os.path.join(package_dir, 'Prompts')

In [15]:
combined_assistant = CombinedAssistant(api_key)

In [16]:
combined_assistant.simple_evaluation_pipeline(search_axis_1 = texts, attributes = attributes)

Attributes extracted
Attributes ['happy, sad, angry, surprised, grateful']
["In the context of None, the happy, sad, angry, surprised, grateful attribute could refer to a set of emotions or states of being experienced by an individual. These attributes may represent different emotional responses that can affect one's mood, behavior, and overall well-being."]
Descriptions extracted, running ratings.
                    Text happy  sad angry surprised grateful
0  This is a funny story    10    5     2         7        1
1    This is a sad story    10  100     0         5        0
2     John broke his leg     0   90    70        10        0


,Text,happy,sad,angry,surprised,grateful
0,This is a funny story,10,5,2,7,1
1,This is a sad story,10,100,0,5,0
2,John broke his leg,0,90,70,10,0
